In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

batch_size = 32
learning_rate = 0.05
log_interval = 10
n_epochs = 5

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

data_dir = "/Users/robinsuxdorf/Documents/Projekte/train_data/"

transform = transforms.Compose([
    # transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# train_data = datasets.MNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor()
# )

# train_loader = DataLoader(
#     dataset=train_data,
#     batch_size=batch_size,
#     shuffle=True
# )

# test_data = datasets.MNIST(
#     root="data",
#     train=False,
#     download=True,
#     transform=ToTensor()
# )

# test_loader = DataLoader(
#     dataset=test_data
# )

In [2]:
import torch.optim as optim
import torch.nn as nn
from symbol_classifier import SymbolClassifier

symbol_classifier = SymbolClassifier()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(symbol_classifier.parameters(), lr=learning_rate)

train_losses: list[float] = []
train_counter: list[int] = []
test_losses: list[float] = []
test_counter: list[int] = []

In [3]:
def train(epoch: int) -> None:
    symbol_classifier.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = symbol_classifier(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f'Train Epoch {epoch}: [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item()}')
            train_losses.append(loss.item())
            train_counter.append((batch_idx * batch_size) + ((epoch - 1) * len(train_loader.dataset)))

In [4]:
def test():
    symbol_classifier.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = symbol_classifier(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)

        print(f'\nTest set: Avg. loss: {test_loss}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset)}%)\n')


In [5]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()


Test set: Avg. loss: 2.3026793009757998, Accuracy: 953/10000 (9.529999732971191%)

Train Epoch 1: [0/60000] Loss: 2.3038692474365234
Train Epoch 1: [320/60000] Loss: 2.3031516075134277
Train Epoch 1: [640/60000] Loss: 2.3009562492370605
Train Epoch 1: [960/60000] Loss: 2.303913116455078
Train Epoch 1: [1280/60000] Loss: 2.3002705574035645
Train Epoch 1: [1600/60000] Loss: 2.30000376701355
Train Epoch 1: [1920/60000] Loss: 2.3052830696105957
Train Epoch 1: [2240/60000] Loss: 2.3036460876464844
Train Epoch 1: [2560/60000] Loss: 2.30016827583313
Train Epoch 1: [2880/60000] Loss: 2.300318479537964
Train Epoch 1: [3200/60000] Loss: 2.3016817569732666
Train Epoch 1: [3520/60000] Loss: 2.301354169845581
Train Epoch 1: [3840/60000] Loss: 2.304392099380493
Train Epoch 1: [4160/60000] Loss: 2.302966356277466
Train Epoch 1: [4480/60000] Loss: 2.299880266189575
Train Epoch 1: [4800/60000] Loss: 2.300579309463501
Train Epoch 1: [5120/60000] Loss: 2.303640842437744
Train Epoch 1: [5440/60000] Loss:

In [6]:
torch.save(symbol_classifier.state_dict(), "model.pt")